In [ ]:
training_file_path = '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.data.xml'
gold_file_path =  '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.gold.key.txt'

training_file_path = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/semeval2007.data.xml'
gold_file_path = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/semeval2007.gold.key.txt'

training_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/semeval2013.data.xml'
gold_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/semeval2013.gold.key.txt'
fine_senses_vocab_path = '../resources/semcor.vocab.WordNet.json'
input_vocab_path = '../resources/semcor.input.vocab.json'
input_antivocab_path = '../resources/semcor.leftout.vocab.json'
embedding_size = 32
batch_size = 64
LEARNING_RATE = 0.01
N_EPOCHS = 10
PADDING_SIZE = 50
print_model = True

In [ ]:
import models
import utils
import generators
import generatorsCopy

from tensorflow.random import set_random_seed
set_random_seed(42)
import tensorflow.keras as K
#import keras as K
import time
import os
#from tqdm import tqdm
import numpy as np
import time
import os
from tqdm import  tqdm

In [ ]:
#loading dict
senses = utils.json_vocab_reader(fine_senses_vocab_path)
inputs, antivocab = utils.json_vocab_reader(input_vocab_path, input_antivocab_path)
output_vocab = utils.vocab_merge(senses, inputs)
reverse_output_vocab =  dict((v, k) for k, v in output_vocab.items())

K.backend.clear_session()

# Build Model

In [ ]:
#model = models.Basic(test=2)
BasicModelNetwork = models.Basic(vocab_size = len(output_vocab),
                                embedding_size = embedding_size,
                                hidden_size = 32,
                                PADDING_SIZE = PADDING_SIZE,
                                LEARNING_RATE = LEARNING_RATE,
                                INPUT_DROPOUT = 0.2,
                                LSTM_DROPOUT = 0.45,
                                RECURRENT_DROPOUT = 0.35,
                                N_EPOCHS = N_EPOCHS)

if print_model is True:
    BasicModelNetwork.summary()


In [14]:
train_generator = generators.Basic(batch_size = 64,
                                training_file_path = training_file_path,
                                gold_file_path = gold_file_path,
                                antivocab = antivocab,
                                output_vocab = output_vocab,
                                PADDING_SIZE = PADDING_SIZE)

validation_generator = generators.Basic(batch_size = 64,
                                         training_file_path = training_file_path_dev,
                                         gold_file_path = gold_file_path_dev,
                                         antivocab = antivocab,
                                         output_vocab = output_vocab,
                                         PADDING_SIZE = PADDING_SIZE)


In [17]:
if not os.path.exists('../resources/logging'):
    os.mkdir('../resources/logging')
model_name = time.strftime('%Y-%m-%d_%H:%M:%S_%z')
cbk = K.callbacks.TensorBoard('../resources/logging/keras_model_'+model_name)

early_stopping = K.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              verbose=2, mode='auto')
csv_logger = K.callbacks.CSVLogger('../resources/logging/keras_model_'+model_name+'.log')
model_checkpoint = K.callbacks.ModelCheckpoint(filepath = '../resources/logging/keras_model_'+model_name+'.h5',
                                               monitor='val_precision',
                                               verbose=2,
                                               save_best_only=True,
                                               save_weights_only=True,
                                               mode='auto', period=1)





W0906 14:18:20.617034 140072756623104 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


In [18]:
# for epoch in range(1, epochs+1):
#     with tqdm(total=epochs) as epoch_bar:
#         c1, c2 = [0,0], [0,0]
#         epoch_bar.set_description("Epoch {}/{} loss: {:.3f}\tacc: {:.3f}\tval_loss: {:.3f}\tval_acc: {:.3f}".format(epoch, epochs, *c1, *c2))
#         for batch_x, batch_y in train_generator.__getitem__():
#             c1 = BasicModelNetwork.train_on_batch(batch_x, batch_y)
#             epoch_bar.set_description("Epoch {}/{} loss: {:.3f}\tacc: {:.3f}\tval_loss: {:.3f}\tval_acc: {:.3f}".format(epoch, epochs, *c1, *c2))

#         for batch_x, batch_y in validation_generator.__getitem__():
#             c2 = BasicModelNetwork.test_on_batch(batch_x, batch_y)
#             epoch_bar.set_description("Epoch {}/{} loss: {:.3f}\tacc: {:.3f}\tval_loss: {:.3f}\tval_acc: {:.3f}".format(epoch, epochs, *c1, *c2))


In [19]:
fmt = "Epoch {}/{} loss: {:.3f}\tacc: {:.3f}\tval_loss: {:.3f}\tval_acc: {:.3f}"

In [20]:
epochs = 2

In [21]:
for epoch in range(1, epochs+1):
    with tqdm(total=epochs) as epoch_bar:
    
        c1, c2 = [0,0], [0,0]
        epoch_bar.set_description(fmt.format(epoch, epochs, *c1, *c2))
        
        train_stats = {"loss": [], "acc": []}
        
        for batch_x, batch_y in train_generator.__getitem__():
            c1 = BasicModelNetwork.train_on_batch(batch_x, batch_y)
            epoch_bar.set_description(fmt.format(epoch, epochs, *c1, *c2))
            train_stats['loss'].append(c1[0])
            train_stats['acc'].append(c1[1])
            
        c1 = np.mean(train_stats['loss']), np.mean(train_stats['acc'])
        epoch_bar.set_description(fmt.format(epoch, epochs, *c1, *c2))
        
        val_stats = {"loss": [], "acc": []}
        
        for batch_x, batch_y in validation_generator.__getitem__():
            c2 = BasicModelNetwork.test_on_batch(batch_x, batch_y)
            epoch_bar.set_description(fmt.format(epoch, epochs, *c1, *c2))
            val_stats['loss'].append(c2[0])
            val_stats['acc'].append(c2[1])
        
        c2 = np.mean(val_stats['loss']), np.mean(val_stats['acc'])
        epoch_bar.set_description(fmt.format(epoch, epochs, *c1, *c2))
            
        


Epoch 1/2 loss: 11.156	acc: 0.466	val_loss: 10.808	val_acc: 0.490:   0%|          | 0/2 [00:07<?, ?it/s]
Epoch 2/2 loss: 10.504	acc: 0.596	val_loss: 9.740	val_acc: 0.489:   0%|          | 0/2 [00:02<?, ?it/s]


In [ ]:
from six import next

patience = 4
best_loss = 1e6
rounds_without_improvement = 0

for epoch_nb in range(nb_of_epochs):
    losses_list = list()
    for batch in range(nb_of_batches):
        x, y = next(train_data_gen)
        losses_list.append(model.train_on_batch(x, y))
    mean_loss = sum(losses_list) / len(losses_list)

    if mean_loss < best_loss:
        best_loss = mean_loss
        rounds_witout_improvement = 0
    else:
        rounds_without_improvement +=1

    if rounds_without_improvement == patience:
        break

In [ ]:
from six import next

patience = 4
best_loss = 1e6
rounds_without_improvement = 0

for epoch_nb in range(nb_of_epochs):
    losses_list = list()
    for batch in range(nb_of_batches):
        x, y = next(train_data_gen)
        losses_list.append(model.train_on_batch(x, y))
    mean_loss = sum(losses_list) / len(losses_list)

    if mean_loss < best_loss:
        best_loss = mean_loss
        rounds_witout_improvement = 0
    else:
        rounds_without_improvement +=1

    if rounds_without_improvement == patience:
        break

In [ ]:
train_generator = generatorsCopy.get(batch_size = 64,
                                training_file_path = training_file_path,
                                gold_file_path = gold_file_path,
                                antivocab = antivocab,
                                output_vocab = output_vocab,
                                PADDING_SIZE = PADDING_SIZE)

validation_generator = generatorsCopy.get(batch_size = 64,
                                         training_file_path = training_file_path_dev,
                                         gold_file_path = gold_file_path_dev,
                                         antivocab = antivocab,
                                         output_vocab = output_vocab,
                                         PADDING_SIZE = PADDING_SIZE)

In [ ]:
for i, j in validation_generator:
    print(i)

In [ ]:
BasicModelNetwork.fit_generator(train_generator, 
                                steps_per_epoch=None,
                                epochs=1, 
                                verbose=3,
                                callbacks=None,
                                validation_data=validation_generator,
                                validation_steps=None,
                                validation_freq=1,
                                class_weight=None,
                                max_queue_size=10,
                                workers=-1, 
                                use_multiprocessing=False,
                                shuffle=False,
                                initial_epoch=0)

In [ ]:
for epoch in range(epochs):
    with tqdm(total=len(train_generator)) as pbar:
        for batch_x, batch_y in train_generator.__getitem__():
            c1 = BasicModelNetwork.train_on_batch(batch_x, batch_y)
            pbar.set_description("Epoch {}/{}\tloss: {:.3f} \tacc: {:.3f}".format(epoch, epochs, *c1))

    with tqdm(total=len(validation_generator)) as pbar:
        for batch_x, batch_y in validation_generator.__getitem__():
            c2 = BasicModelNetwork.test_on_batch(batch_x, batch_y)
            pbar.set_description("Epoch {}/{}\tval_loss: {:.3f} \tval_acc: {:.3f}".format(epoch, epochs, *c2))
